# Supervised HMM 

In [35]:
import os
import sys
sys.path.append('../libraries/')
from data_loader import data_loader
from behavior_loader import bsoid_loader

import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
data_dir = '../processed_data'

## Load neural data

In [37]:
filename = 'm1_fr0.1hz_30hz_0s_7200s_0.1s_bins_nooverlap.sav'
dl = data_loader(data_dir, filename)
data = dl.load_data()
_, D = data.shape


Spike times are binned into shape for hmm: (36000, 46)


## Load behavior targets

In [39]:
filename = 'ag25290_day2_iter13'
bl = bsoid_loader(data_dir, filename)
_, _, smoothed_predictions = bl.main()
frame_rate = 60
delay = 6.8503
behavior_start = int(delay * frame_rate)  # start of behavior
smoothed_predictions_1h = smoothed_predictions[behavior_start:(behavior_start+3600*frame_rate)]

mouse stayed in nest for 0.28108749470484795% of time
File #0 (a 6 body parts by 984391 frames) has 30 classes


In [41]:
states, state_cnts = np.unique(smoothed_predictions_1h, return_counts=True)
print(state_cnts)

[ 2783  3236   388    29  1027   750   654  1772 13852 53834 18433  5738
    62   103   453  3654  8784  2133  6194  9370   233  1972  7292  2285
  6491  4761  5077 47599   773  6268]


In [5]:
data_rate = 10 # samples per minute
# For now simply use the exact time stamp for data and targets to match
targets = smoothed_predictions_1h[::int(frame_rate/data_rate)]
targets = targets + 1
names_rearrange = ['torso groom', 'investigate type 0', 'unsure', 'wall rear type 1', 'rear',
         'unsure', 'wall rear type 2', 'investigate type 1', 'investigate type 2', 'contra-itch',
         'investigate type 3', 'sleep/scrunch type 1', 'sleep/scrunch type 2', 'wall rear type 3', 'contra-body groom',
         'face groom type 1', 'dive/scrunch', 'head groom', 'ipsi-orient', 'ipsi-investigate',
         'face groom type 2', 'ipsi-body groom', 'ipsi-itch type 1', 'ipsi-itch type 2', 'face groom type 3', 
         'paw groom', 'locomotion', 'contra-forepaw orient', 'contra-orient']
# Number of targets, i.e., number of hmm states to set
K = len(np.unique(targets))

## hmm class

In [33]:
np.random.seed(10)
a = np.random.rand(20, 4) * 100
# print(a)
a_mean = np.mean(a, axis=0)
# print(a_mean)
a_cov = (a - a_mean).T @ (a - a_mean) / (20)
# eig_val, eig_vec = np.linalg.eig(np.cov(a.T, bias=False))
# print(np.max(eig_val))
eig_val, eig_vec = np.linalg.eig(a_cov)
print(np.max(eig_val))
# print(eig_vec)


1185.4725971030498


In [ ]:
class hmm:
    def __init__(self, K: int, D: int) -> None:
        self.K = K
        self.D = D
        # self.initial_state
        self.transitions = np.zeros((self.K, self.K))
        self.emissions_mean = np.zeros((self.K, self.D))
        self.emissions_cov = np.zeros((self.K, self.D, self.D))
    def fit(self, data: np.array, targets: np.array) -> None:
        unique_targets = np.unique(targets)
        for i, target in enumerate(unique_targets):
            target_indcs = np.where(targets == target)[0]
            target_data = data[target_indcs, :]
            target_data_mean = np.mean(target_data, axis=0)
            self.emissions_mean[i, :] = target_data_mean
            self.emissions_cov[i, :, :] = np.cov((target_data - target_data_mean).T, bias=True) #(target_data - target_data_mean).T @ (target_data - target_data_mean) / len(target_indcs)
            trans_target_indcs = target_indcs + 1
            if trans_target_indcs[-1] == len(targets):
                trans_target_indcs = trans_target_indcs[:-1]
            trans_targets = targets[trans_target_indcs]
            for j, target in enumerate(unique_targets):
                self.transitions[i][j] = len(np.where(trans_targets == target)[0]) / len(trans_targets)
    def predict(self, data: np.array) -> np.array:
        targets_list = []
        
        pass
    def f1_score(self, data, targets):

        pass